In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!scp /content/drive/MyDrive/ocr/data.zip "/content/data.zip"
!unzip "/content/data.zip" -d "/content/"


In [ ]:
!apt install tesseract-ocr
!pip install pytesseract

!pip install Pillow
!pip install easyocr
!pip install boto3

In [ ]:
!apt install libtesseract-dev

In [ ]:
import pytesseract
from PIL import Image
from easyocr import Reader
import boto3


reader = Reader(['en'])

access_key = None # aws console keys girilecek
secret_access_key = None

textract_client = boto3.client('textract',
                               aws_access_key_id=access_key,
                               aws_secret_access_key = secret_access_key,
                               region_name='us-east-1')


def read_text_tesseract(image_path):

  text = pytesseract.image_to_string(Image.open(image_path), lang='eng')
  return text

def read_text_easyocr(image_path):
  text = ''
  results = reader.readtext(Image.open(image_path))
  for result in results:
    text = text + result[1] +  ' '

  text = text[:-1]
  return text

def read_text_textract(image_path):

  with open(image_path, 'rb') as im:
    response = textract_client.detect_document_text(Document={'Bytes':im.read()})

  text = ''
  for item in response['Blocks']:
    if item['BlockType'] == 'LINE':
      text = text + item['Text'] + ' '

  text = text[:-1]
  return text


In [ ]:
import os


def jaccard_similarity(sentence1, sentence2):

    set1 = set(sentence1.lower().split())
    set2 = set(sentence2.lower().split())

    intersection_size = len(set1.intersection(set2))
    union_size = len(set1.union(set2))

    similarity = intersection_size / union_size if union_size != 0 else 0.0

    return similarity


score_tesseract = 0
score_easyocr = 0
score_textract = 0
for image_path_ in os.listdir('/content/OCR-Dataset'):
  image_path = os.path.join('/content/OCR-Dataset', image_path_)

  gt = image_path[:-4].replace('_', ' ').lower()

  score_tesseract += jaccard_similarity(gt, read_text_tesseract(image_path).lower().replace('\n', '').replace('!', '').replace('?', '').replace('.', ''))
  score_easyocr += jaccard_similarity(gt, read_text_easyocr(image_path).lower().replace('\n', '').replace('!', '').replace('?', '').replace('.', ''))
  score_textract += jaccard_similarity(gt, read_text_textract(image_path).lower().replace('\n', '').replace('!', '').replace('?', '').replace('.', ''))

print('score tesseract:', score_tesseract / 100)
print('score_easyocr:', score_easyocr / 100)
print('score_textract:', score_textract / 100)
